# Task 3 — PS-PL GPIO Calculator (Notebook)

This notebook contains the Task 3 PS-side driver code for the **PYNQ-Z2** board that communicates with the PL via EMIO GPIO (52-bit protocol).

**Notes**
- The notebook **does not execute the hardware calls** here; it's a code artifact you can run on your PYNQ board (or edit first).
- It includes a debug-enabled `execute_calculation()` implementation and a small smoke-test loop.
- If you run the notebook on your board, it will attempt to import `pynq.Overlay` and use `MMIO` — make sure you run this on the PYNQ target (or comment out the hardware parts when testing on a host).


In [ ]:
# TASK 3 – PS-PL GPIO CALCULATOR
#
# hw:
#   - ZYNQ PS + PL
#   - EMIO GPIO, 52-bit wide
#   - RTL: task3.v
#
# GPIO bit layout (LSB → MSB):
#   [23:0]   : inputs passthrough (in0-in1-in2 packed)
#   [47:24]  : output
#   [48]     : done
#   [51:49]  : opcode
#
# author: Alp Bolukbasi
#
# NOTE: Run this notebook on the PYNQ board. The hardware-specific imports (pynq) will fail on a normal PC.

from pynq import Overlay, MMIO
from dataclasses import dataclass
import time

# Load overlay (ensure task3.bit present on the board filesystem)
overlay = Overlay("task3.bit")

# GPIO register addresses
GPIO_BASE = 0xe000a000
DATA2_RO  = 0x068
DATA3_RO  = 0x06c
DATA2     = 0x048
DATA3     = 0x04c
DIRM2     = 0x284
DIRM3     = 0x2c4
OEN2      = 0x288
OEN3      = 0x2c8

# Protocol constants
RESULT_LSB = 24
DONE_BIT   = 48
OPCODE_LSB = 49
MASK_24 = (1 << 24) - 1
MASK_3  = (1 << 3)  - 1

# MMIO window
mmio = MMIO(GPIO_BASE, 0x1000)

# Configure banks (PS drives EMIO[0..23], PL drives result bits)
mmio.write(DIRM2, 0x00ffffff)
mmio.write(OEN2,  0x00ffffff)
opcode_mask = (0x7 << (49 - 32))
mmio.write(DIRM3, opcode_mask)
mmio.write(OEN3,  opcode_mask)

@dataclass
class CalcResult:
    opcode: int
    result: int
    done: bool

def to_signed(value, bits):
    """Convert two's complement unsigned value to Python signed int."""
    sign = 1 << (bits - 1)
    return (value ^ sign) - sign

def to_unsigned(value, bits):
    return value & ((1 << bits) - 1)

def execute_calculation(opcode, in0, in1, in2, debug=False):
    """
    Execute one calculation on the PL via EMIO GPIO.
    Inputs in0,in1,in2 are treated as 8-bit signed integers (Python ints).
    Returns CalcResult(opcode, signed_result, done_flag)
    """

    # Ensure inputs are 8-bit signed then convert to transport unsigned 8-bit pattern
    in0_s = to_signed(in0 & 0xff, 8)
    in1_s = to_signed(in1 & 0xff, 8)
    in2_s = to_signed(in2 & 0xff, 8)

    in0_u = to_unsigned(in0_s, 8)
    in1_u = to_unsigned(in1_s, 8)
    in2_u = to_unsigned(in2_s, 8)

    # Pack inputs
    inputs_packed = (in2_u << 16) | (in1_u << 8) | in0_u

    # Build 52-bit word (opcode at bits [51:49])
    word_out = (inputs_packed & MASK_24) | ((opcode & MASK_3) << OPCODE_LSB)

    if debug:
        print("WRITE -> opcode={}, in0={}, in1={}, in2={}".format(opcode, in0_s, in1_s, in2_s))
        print("inputs_packed = 0x{:06x}".format(inputs_packed))
        print("word_out      = 0x{:013x}".format(word_out))

    # Write to GPIO
    mmio.write(DATA2, word_out & 0xffffffff)
    mmio.write(DATA3, (word_out >> 32) & 0xffffffff)

    # Poll for DONE
    start_time = time.time()
    while (time.time() - start_time) < 2.0:
        low_bits  = mmio.read(DATA2_RO)
        high_bits = mmio.read(DATA3_RO)

        # Stitch result explicitly: low byte of result is low_bits[31:24], high 16 bits are high_bits[15:0]
        low_result_byte = (low_bits >> 24) & 0xff
        high_result_16  = high_bits & 0xffff
        raw_result = (high_result_16 << 8) | low_result_byte

        # DONE is bit 48 -> high_bits bit (48-32)=16
        done = (high_bits >> (48 - 32)) & 0x1

        if debug:
            full_word = (high_bits << 32) | (low_bits & 0xffffffff)
            print("READ  low_bits  = 0x{:08x}".format(low_bits))
            print("READ  high_bits = 0x{:08x}".format(high_bits))
            print("READ  full_word = 0x{:016x}".format(full_word))
            print("raw_result (24) = 0x{:06x}".format(raw_result))

        if done:
            signed_result = to_signed(raw_result, 24)
            return CalcResult(opcode, signed_result, True)

    return CalcResult(opcode, 0, False)

# Example smoke tests (enable debug for the problematic vector)
test_cases = [
    (0,  3,  5,  7),
    (1, 10,  4,  5),
    (2,  8,  1,  6),
    (3, 15,  3,  1),
    (4,  9,  0,  0),
    (5,  6,  2,  4),
    (6, 12,  3,  2),
    (7,  3,  2,  1),
    (7,  5, 10,  3)  # problematic vector
]

print("Task 3 ALU verification (all opcodes):\\n")
for op, i0, i1, i2 in test_cases:
    dbg = (op == 7 and i0 == 5 and i1 == 10 and i2 == 3)
    res = execute_calculation(op, i0, i1, i2, debug=dbg)
    status = "SUCCESS" if res.done else "TIMEOUT"
    print("opcode {}: in0={}, in1={}, in2={} -> result={} | status={}".format(op, i0, i1, i2, res.result, status))
